# PREAMBLE
<script
  src="https://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML"
  type="text/javascript">
</script>

In [ ]:
import numpy as np
import pandas as pd
import numpy.linalg as la
import seaborn as sns
sns.set()
sns.set_style("ticks")
from validphys.api import API
from validphys.loader import Loader
from matplotlib import pyplot as plt
from matplotlib import cm
l = Loader()
import yaml

# Definition of the input

In [ ]:
fit="221207-ern-002"

theory_max = 236
theory_mid = 205
theory_min = 201

alphas_step_size = 0.003
alphas_central = 0.119

covmat_scaling_factor = 1

# COMPUTATION OF $\alpha_s$

In [ ]:
fitpath = API.fit(fit=fit).path 
filterpath = fitpath / 'filter.yml'
import yaml
with open(filterpath) as f:
    filterfile = yaml.safe_load(f)
pdf_ori=filterfile['theorycovmatconfig']['pdf']

In [ ]:
common_dict = dict(dataset_inputs={"from_": "fit"},
            fit=fit,
            use_cuts="fromfit",
            metadata_group="nnpdf31_process",)

In [ ]:
#Inputs for central theory
inps_central = dict(theoryid=theory_mid,pdf=pdf_ori,**common_dict)

In [ ]:
#Inputs for plus theory
inps_plus = dict(theoryid=theory_max,pdf=pdf_ori,**common_dict)

In [ ]:
#Inputs for minus theory
inps_minus = dict(theoryid=theory_min,pdf=pdf_ori,**common_dict)

In [ ]:
#Inputs for central theory
inps_central_fit = dict(theoryid=theory_mid,pdf={"from_": "fit"},**common_dict)

In [ ]:
#Experimental covariance matrix
C = API.groups_covmat(**inps_central)

In [ ]:
dsindex=API.groups_index(**inps_central)

In [ ]:
datth_central = API.group_result_table_no_table(**inps_central)

In [ ]:
datth_plus = API.group_result_table_no_table(**inps_plus)

In [ ]:
datth_minus = API.group_result_table_no_table(**inps_minus)

In [ ]:
datth_central_fit = API.group_result_table_no_table(**inps_central_fit)

In [ ]:
dat_central=datth_central["data_central"]
th_replicas_fit=datth_central_fit.iloc[:,2:].to_numpy()

Computation of Eqs.(3.37)-(3.38) in [arXiv:2105.05114](https://arxiv.org/pdf/2105.05114.pdf)

In [ ]:
beta_tilde = np.sqrt(covmat_scaling_factor)*(alphas_step_size/np.sqrt(2))*np.array([1,-1])
S_tilde = beta_tilde@beta_tilde

In [ ]:
delta_plus  = (np.sqrt(covmat_scaling_factor)/np.sqrt(2))*(datth_plus["theory_central"] - datth_central["theory_central"]).to_numpy()
delta_minus = (np.sqrt(covmat_scaling_factor)/np.sqrt(2))*(datth_minus["theory_central"] - datth_central["theory_central"]).to_numpy()
beta = [delta_plus,delta_minus]
S_hat = beta_tilde@beta

In [ ]:
S = np.outer(delta_plus,delta_plus)+np.outer(delta_minus,delta_minus)
S = pd.DataFrame(S,index=dsindex,columns=dsindex)
S = pd.DataFrame(S.values, index=C.index, columns=C.index)

In [ ]:
invcov = la.inv(C+S)

In [ ]:
# Different from the prediction of the mean PDF (i.e. replica0)
mean_prediction = np.mean(th_replicas_fit[:],axis=1)

In [ ]:
X = np.zeros_like(C.values)
for i in range(th_replicas_fit.shape[1]):
    X += np.outer((th_replicas_fit[:,i]-mean_prediction),(th_replicas_fit[:,i]-mean_prediction))
X *= 1/th_replicas_fit.shape[1]

Final result

In [ ]:
# BUG: dat_central should become average over data replicas
delta_T_tilde = S_hat@invcov@(dat_central-mean_prediction)
P_tilde = S_hat.T@invcov@X@invcov@S_hat + (S_tilde - S_hat.T@invcov@S_hat)
pred = alphas_central + delta_T_tilde
unc  = np.sqrt(P_tilde)

In [ ]:
aa = pd.read_csv(
    fitpath / 'tables/datacuts_theory_theorycovmatconfig_theory_covmat_custom.csv', 
    sep='\t',encoding='utf-8', index_col=2,header=3,skip_blank_lines=False,
);
if np.allclose(S.to_numpy(),aa.to_numpy()[:,2:].astype('float64')):
    print(rf"Prediction for $\alpha_s$: {pred:.5f} +/- {unc:.5f}")
else:
    print("Reconstructed theory covmat, S, is note the same as the stored covmat!")